# Main

In [129]:
'''Main Libraries'''
import numpy as np
import pandas as pd

'''Datetime'''
import time
from datetime import datetime
from dateutil.parser import parse

'''Modeling'''
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score

'''Visualization'''
import matplotlib.pyplot as plt
import graphviz
from graphviz import Source
from sklearn.tree import export_graphviz
from IPython.display import Image

## Uplaod data 
Converting from json to pandas dataframe

In [2]:
data = pd.read_json (r'song.json')

### Data Preview

In [3]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


## Date-time formatting

In [4]:
data['user_sign_up_date'] =  pd.to_datetime(data['user_sign_up_date'])
data['time_played'] = pd.to_datetime(data['time_played'])
data['day_played'] = pd.to_datetime(data['time_played'].apply(lambda x: str(x).split()[0]))
data['time_of_day_played'] = data['time_played'].apply(lambda x: str(x).split(' ')[1])
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played,day_played,time_of_day_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35,2015-06-11,21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19,2015-06-06,16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29,2015-06-14,02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10,2015-06-08,12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,2015-06-28,14:57:00


# What are the top 3 and the bottom 3 states in terms number of users?

In [5]:
x = data.groupby(['user_state'])['user_id'].agg({'users_per_state': lambda x: len(x.unique()),
                                                'total_acitivity_per_state': lambda x: len(x)})

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  


In [6]:
print('States with the most number of users')
print('-------------------------------------')
print(x['users_per_state'].nlargest(3))
print('States with the most number of users')
print('-------------------------------------')
print(x['users_per_state'].nsmallest(3))

States with the most number of users
-------------------------------------
user_state
New York      23
California    21
Texas         15
Name: users_per_state, dtype: int64
States with the most number of users
-------------------------------------
user_state
Arizona        1
Connecticut    1
Idaho          1
Name: users_per_state, dtype: int64


# What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically define user engagement. What the CEO cares about here is in which states users are using the product a lot/very little.

In [7]:
x['total_acitivity_per_state'].nlargest(3)

user_state
New York      469
California    425
Texas         230
Name: total_acitivity_per_state, dtype: int64

In [8]:
x['total_acitivity_per_state'].nsmallest(3)

user_state
Kansas          8
Connecticut    16
New Mexico     17
Name: total_acitivity_per_state, dtype: int64

# The CEO wants to send a gift to the first user who signed-up for each state. That is, the first user who signed-up from California, from Oregon, etc. Can you give him a list of those users?

In [9]:
df = data.groupby(['user_state'])['user_sign_up_date'].min()
ids = {}
for i in data['user_state'].unique():
    df1 = data[data['user_state'] == i]
    df2 = df1[df1['user_sign_up_date'] == df[i]]['user_id'].iloc[0]
    ids[i] = df2
print('first user for each state')
ids

first user for each state


{'Louisiana': 50,
 'Ohio': 3,
 'New Jersey': 6,
 'Illinois': 45,
 'Texas': 7,
 'Rhode Island': 174,
 'Oregon': 1,
 'California': 39,
 'New York': 19,
 'Wisconsin': 32,
 'Tennessee': 70,
 'Maryland': 18,
 'Michigan': 13,
 'Pennsylvania': 11,
 'Missouri': 85,
 'Alabama': 5,
 'Oklahoma': 119,
 'Washington': 125,
 'Idaho': 165,
 'Kentucky': 34,
 'Arkansas': 78,
 'Connecticut': 127,
 'Colorado': 173,
 'Georgia': 20,
 'Utah': 29,
 'North Carolina': 2,
 'Alaska': 106,
 'Mississippi': 23,
 'Florida': 41,
 'West Virginia': 60,
 'South Carolina': 64,
 'Massachusetts': 15,
 'Virginia': 142,
 'Indiana': 102,
 'Iowa': 178,
 'Minnesota': 8,
 'North Dakota': 135,
 'New Mexico': 4,
 'Nebraska': 134,
 'Arizona': 105,
 'Kansas': 177}

# Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next.
That is, if, for instance, a user is currently listening to “Eight Days A Week“, which song has the highest probability of being played right after it by the same user? This is going to be V1 of a song recommendation model.

## Feature Engineering 

### Number of times song played by each user

In [10]:
data['song_played_times'] = data.groupby(['song_played','user_id'])['day_played'].transform('count')

### Number of times song played by all users overall

In [11]:
data['song_played_times_overall_day'] = data.groupby(['song_played'])['day_played'].transform('count')

### Number of times song played by each state

In [12]:
data['song_played_times_loc'] = data.groupby(['user_state'])['song_played'].transform('count')

### Order in which the song was played by each user by day

In [13]:
data['dummy'] = 1
data['song_played_order'] = data.sort_values(by = 'time_played').groupby(['user_id','day_played'])['dummy'].transform('cumsum').apply(lambda x: x)

In [14]:
data.drop('dummy', axis = 1, inplace = True)
data[data['user_id'] == 3].sort_values(by = 'time_played').head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played,day_played,time_of_day_played,song_played_times,song_played_times_overall_day,song_played_times_loc,song_played_order
3669,TGATMXQGDZ,3,Ohio,2015-05-01,A Day In The Life,2015-06-01 19:18:04,2015-06-01,19:18:04,1,189,209,1
1245,NJSUBWRMQQ,3,Ohio,2015-05-01,Get Back,2015-06-01 20:31:49,2015-06-01,20:31:49,3,279,209,2
179,PBXOQQBWEA,3,Ohio,2015-05-01,Lucy In The Sky With Diamonds,2015-06-04 01:08:14,2015-06-04,01:08:14,2,113,209,1
3862,RKKESLRFSH,3,Ohio,2015-05-01,Hello Goodbye,2015-06-06 14:43:29,2015-06-06,14:43:29,1,161,209,1
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19,2015-06-06,16:49:19,1,99,209,2


In [15]:
data['time_of_day_played'] = pd.to_datetime(data['time_of_day_played'])
data['time_of_day_played'] = data['time_of_day_played'].dt.hour*3600 + data['time_of_day_played'].dt.minute*60 + data['time_of_day_played'].dt.second

In [16]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played,day_played,time_of_day_played,song_played_times,song_played_times_overall_day,song_played_times_loc,song_played_order
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35,2015-06-11,78695,1,120,105,1
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19,2015-06-06,60559,1,99,209,2
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29,2015-06-14,7889,2,163,117,1
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10,2015-06-08,44770,1,6,149,1
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,2015-06-28,53820,1,8,117,1


### difference between sign up day and first played

In [192]:
data['first_played'] = data.groupby('user_id')['day_played'].transform('min')
data['sign_up_to_first_played'] = (data['first_played'] - data['day_played']).astype(int)

## Model

In [193]:
df = data.drop(['user_sign_up_date','time_played',
                'day_played','song_played','first_played','id'], axis = 1)
le = preprocessing.LabelEncoder()
y = data['song_played']
le.fit(y)
y = le.transform(y)

### Training

In [194]:
X_train, X_test, y_train, y_test = train_test_split(pd.get_dummies(df), 
                                                    y, random_state=42, test_size = 0.33)
rfc =  AdaBoostClassifier(n_estimators=100, random_state=42,
                         base_estimator = DecisionTreeClassifier(max_depth = 10))
rfc.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=10,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                            

### Predicting

In [195]:
y_pred = rfc.predict(X_test)

In [196]:
rfc.score(X_test, y_test)

0.8333333333333334

In [198]:
predictions = pd.DataFrame()
predictions['Guessed Song'] = le.inverse_transform(y_pred)
predictions['Actual Song'] = le.inverse_transform(y_test)
predictions.to_csv('predictions.csv')

### Score
How would you set up a test to check whether your model works well?

In [199]:
f1_score(y_test, y_pred, average='weighted')

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.8308758084166559